In [4]:
import onnxruntime as ort
import numpy as np
import torch
import time
from pathlib import Path

print("="*70)
print("🧪 ONNX Runtime GPU 配置诊断（修复版）")
print("="*70)

# ==================== 1. PyTorch GPU 检测 ====================
print("\n1️⃣ PyTorch GPU 检测:")
print(f"   CUDA 可用: {torch.cuda.is_available()}")
print(f"   CUDA 版本: {torch.version.cuda}")
print(f"   GPU 数量: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"\n   GPU {i}:")
        print(f"      名称: {props.name}")
        print(f"      显存: {props.total_memory / 1e9:.2f} GB")
        print(f"      Compute Capability: {props.major}.{props.minor}")

# ==================== 2. ONNX Runtime 可用提供者 ====================
print("\n\n2️⃣ ONNX Runtime 可用执行提供者:")
available_providers = ort.get_available_providers()
print(f"   所有提供者: {available_providers}")

if 'CUDAExecutionProvider' in available_providers:
    print("   ✅ CUDAExecutionProvider 可用")
else:
    print("   ❌ CUDAExecutionProvider 不可用")

# ==================== 3. CPU vs GPU 性能对比 ====================
print("\n\n3️⃣ CPU vs GPU 性能对比:")

input_size = (1, 3, 224, 224)
test_input = np.random.randn(*input_size).astype(np.float32)

print(f"\n   测试输入形状: {input_size}")

MODEL_PATH = Path("../../model/clip-image-encoder.onnx")

if not MODEL_PATH.exists():
    print(f"   ⚠️  找不到模型: {MODEL_PATH}")
else:
    print(f"   ✅ 找到模型: {MODEL_PATH}")

    # ========== CPU 推理 ==========
    print("\n   🔵 CPU 推理测试:")
    opts = ort.SessionOptions()
    opts.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

    sess_cpu = ort.InferenceSession(str(MODEL_PATH), opts,
                                    providers=['CPUExecutionProvider'])
    input_name = sess_cpu.get_inputs()[0].name

    # 预热
    for _ in range(3):
        sess_cpu.run(None, {input_name: test_input})

    # 计时
    times_cpu = []
    for _ in range(10):
        start = time.perf_counter()
        sess_cpu.run(None, {input_name: test_input})
        times_cpu.append((time.perf_counter() - start) * 1000)

    cpu_avg = np.mean(times_cpu)
    cpu_std = np.std(times_cpu)
    print(f"      平均延迟: {cpu_avg:.2f} ms ± {cpu_std:.2f} ms")

    # ========== GPU 推理 ==========
    if 'CUDAExecutionProvider' in available_providers:
        print("\n   🟠 GPU 推理测试:")

        # ✅ 简化的配置（兼容）
        providers = [
            ('CUDAExecutionProvider', {
                'device_id': 0,
            }),
            'CPUExecutionProvider'
        ]

        sess_gpu = ort.InferenceSession(str(MODEL_PATH), opts, providers=providers)

        print(f"      实际使用的提供者: {sess_gpu.get_providers()}")

        # 预热
        print("      预热中...")
        for _ in range(5):
            sess_gpu.run(None, {input_name: test_input})

        # 计时
        print("      推理中...")
        times_gpu = []
        for _ in range(10):
            start = time.perf_counter()
            sess_gpu.run(None, {input_name: test_input})
            times_gpu.append((time.perf_counter() - start) * 1000)

        gpu_avg = np.mean(times_gpu)
        gpu_std = np.std(times_gpu)
        print(f"      平均延迟: {gpu_avg:.2f} ms ± {gpu_std:.2f} ms")

        # 加速比
        speedup = cpu_avg / gpu_avg
        print(f"\n   🚀 加速比: {speedup:.2f}x")

        if speedup > 1.5:
            print(f"      ✅ GPU 正常工作！")
        elif speedup > 0.8:
            print(f"      ⚠️  GPU 加速效果一般")
        else:
            print(f"      ❌ GPU 未被正确使用")
    else:
        print("\n   🔴 GPU 不可用")

# ==================== 总结 ====================
print("\n\n" + "="*70)
print("📊 诊断总结")
print("="*70)

if torch.cuda.is_available() and 'CUDAExecutionProvider' in available_providers:
    print("\n✅ GPU 环境配置正常！")
else:
    print("\n❌ GPU 环境有问题")

print("="*70)

🧪 ONNX Runtime GPU 配置诊断（修复版）

1️⃣ PyTorch GPU 检测:
   CUDA 可用: True
   CUDA 版本: 12.8
   GPU 数量: 1

   GPU 0:
      名称: NVIDIA GeForce RTX 5070
      显存: 12.34 GB
      Compute Capability: 12.0


2️⃣ ONNX Runtime 可用执行提供者:


AttributeError: module 'onnxruntime' has no attribute 'get_available_providers'